In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from molecule import Molecule
from pes import PES
from avg_pes import Avg_PES
import numpy as np
from nifty import pvec1d,pmat2d
import matplotlib
import matplotlib.pyplot as plt
from pytc import *
import manage_xyz
from rhf_lot import *
from psiw import *
from nifty import getAllCoords,getAtomicSymbols,click,printcool
import pybel as pb
%matplotlib inline

In [2]:
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:  16                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#


'----------------------------------------------------------------\n'

In [3]:
printcool("build the Lightspeed (pyTC) objecs")
qmmm = QMMM.from_prmtop(
    prmtopfile='../data/system.prmtop',
    inpcrdfile='../data/system.inpcrd',
    qmindsfile='../data/system.qm',
    charge=0.,
    )
geom = geometry.Geometry.build(
    resources=resources,
    qmmm=qmmm,
    basisname='3-21g',
    )
print(geom)
ref = RHF(RHF.default_options().set_values({
    'geometry' : geom, 
    'dft_functional' : 'B3LYP',
    'dft_grid_name' : 'SG0',
    }))
psiw = RHF_LOT.from_options(rhf=ref)


#========================================================#
#|          build the Lightspeed (pyTC) objecs          |#
#========================================================#
Geometry:
  QMMM = True
  -D3  = False
  ECP  = False

QM/MM Details:
  Total atoms   = 2774
  QM atoms      = 56
  Link H atoms  = 0

Molecule: QM/MM Full Molecule
  Natom        =        2774
  Charge       =       0.000
  Multiplicity =       1.000

Molecule: QM/MM QM Molecule
  Natom        =          56
  Charge       =       0.000
  Multiplicity =       1.000

Basis: 3-21g
  nao     =   308
  ncart   =   308
  nprim   =   336
  nshell  =   196
  natom   =    56
  pure?   =    No
  max L   =     1

Basis: cc-pvdz-minao
  nao     =   168
  ncart   =   168
  nprim   =   728
  nshell  =   112
  natom   =    56
  pure?   =   Yes
  max L   =     1



In [6]:
####### =>  Build the pyGSM objects <= #########
# => level of theory <= #
printcool("Build the pyGSM level of theory (LOT) object")
mol=next(pb.readfile('pdb','../data/system.pdb'))
coords = getAllCoords(mol)
atoms = getAtomicSymbols(mol)
geom= manage_xyz.combine_atom_xyz(atoms,coords)
states=[(1,0)]
lot = PyTC.from_options(states=states,job_data={'psiw':psiw},geom=geom)

# => Create PES objects <= #
printcool("Building the PES objects")
pes = PES.from_options(lot=lot,ad_idx=0,multiplicity=1)


#========================================================#
#|     Build the pyGSM level of theory (LOT) object     |#
#========================================================#
 initializing LOT from geom
#========================================================#
#|               Building the PES objects               |#
#========================================================#


In [8]:
# => Molecule <= #
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
TRIC_molecule = Molecule.from_options(fnm='../data/system.pdb',PES=pes,coordinate_type="TRIC")


#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 initializing LOT from geom
 making primitives from options!
 Timings: Build G: 11.988 Eig: 1.454
 making primitive Hessian
 forming Hessian in basis


In [9]:
import timeit  
n=5

def update_coords(M):
    M.update_coordinate_basis()


In [10]:
printcool("Forming coordinate experiment!\n running update coords for nsteps={}".format(n))
total_time= timeit.timeit(stmt='update_coords(TRIC_molecule)',setup="from __main__ import update_coords, TRIC_molecule",number=n)
avg_time = total_time/n
print "total time = ",total_time
print "avg time = ",avg_time

#========================================================#
#|            Forming coordinate experiment!            |#
#|          running update coords for nsteps=5          |#
#========================================================#
 Timings: Build G: 1.411 Eig: 1.838
 Timings: Build G: 8.472 Eig: 2.000
 Timings: Build G: 9.884 Eig: 1.616
 Timings: Build G: 8.714 Eig: 1.735
 Timings: Build G: 8.196 Eig: 1.882
total time =  45.9957242012
avg time =  9.19914484024


In [17]:
print(TRIC_molecule.coord_obj.Prims.nprims_frag)

[364, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9

In [20]:
print(TRIC_molecule.coord_obj.Prims.natoms_frag)

[56, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,